In [1]:
import os
import tensorflow as tf
from tasks.tasks import SvAgreementLM, WordSvAgreementLM, WordSvAgreementVP
from tf2_models.lm_transformer import LmGPT2, ClassifierGPT2
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.lm_lstm import LmLSTM, LmLSTMSharedEmb, ClassifierLSTM
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
from util import constants
from collections import Counter
from tqdm import tqdm
import numpy as np
from tf2_models.metrics import *

MODELS = {"lm_lstm": LmLSTM,
          "lm_gpt2": LmGPT2,
          "lm_lstm_shared_emb": LmLSTMSharedEmb,
          'cl_gpt2': ClassifierGPT2,
          'cl_lstm': ClassifierLSTM}

In [2]:
log_dir = "../logs"
chkpt_dir = "../tf_ckpts"
exp_name = "nol2_batchsumloss"

task = WordSvAgreementLM(task_params=get_task_params(),data_dir='../data')

Vocab len:  10034


In [3]:
model_config = 'lstm_drop30_v2'
model_name = 'lm_lstm_shared_emb'
train_config ='radam_fast'
# Create the Model
model_params = get_model_params(task,model_name, model_config)
print("model_params: ", model_params.__dict__)

model = MODELS[model_name](hparams=get_model_params(task,model_name, model_config))

trainer_params = get_train_params(train_config)

log_dir = os.path.join(log_dir,task.name, model.model_name+"_"+str(model_config)+"_"+str(trainer_params.learning_rate)+"_"+exp_name)
ckpt_dir = os.path.join(chkpt_dir,task.name, model.model_name+"_"+str(model_config)+"_"+str(trainer_params.learning_rate)+"_"+exp_name)

print(log_dir)

trainer = Trainer(task=task,
                model=model,
                train_params=get_train_params('radam_fast'),
                log_dir=log_dir,
                ckpt_dir=ckpt_dir)

trainer.restore()

model config: lstm_drop30_v2
{'hidden_dim': 512, 'embedding_dim': 512, 'depth': 2, 'hidden_dropout_rate': 0.3, 'input_dropout_rate': 0.0, 'initializer_range': 0.1}
model_params:  {'embedding_dim': 512, 'hidden_dim': 512, 'input_dim': 10036, 'output_dim': 10036, 'depth': 2, 'hidden_dropout_rate': 0.3, 'input_dropout_rate': 0.0, 'initializer_range': 0.1}
model config: lstm_drop30_v2
{'hidden_dim': 512, 'embedding_dim': 512, 'depth': 2, 'hidden_dropout_rate': 0.3, 'input_dropout_rate': 0.0, 'initializer_range': 0.1}
../logs/word_sv_agreement_lm/lm_lstm_shared_emb_em-512_h-512_d-2_hdrop-0.3_indrop-0.0_lstm_drop30_v2_0.001_nol2_batchsumloss
Model: "lm_lstm_shared_emb"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (SharedEmbeddings) multiple                  5138432   
_________________________________________________________________
dropout (Dropout)            multiple                  0        

In [4]:
trainer.model.evaluate(task.test_dataset, steps=100)

100/100 [==============================] - 7s 72ms/step - loss: 4.4992 - batch_masked_sequence_loss: 4.4992 - masked_batch_perplexity: 90.3702 - masked_perplexity: 128.5419 - accuracy: 0.0600 - accuracy_top2: 0.0804 - accuracy_top5: 0.1069


[4.499240121841431,
 4.49924,
 90.37023,
 128.54193,
 0.06004137,
 0.080423586,
 0.10690875]

In [5]:
for x,y in task.test_dataset:
    print(len(x))
    mask = tf.cast(y > 0, dtype=tf.float32)
    logits = model(x)
    print(logits.shape)
    logits = tf.reshape(logits, (-1, logits.shape[-1]))
    targets = tf.reshape(y, (-1, 1))
    mask = tf.reshape(mask, (-1, 1))
    correct = tf.cast(tf.argmax(model(x), axis=-1) == y, dtype=tf.float32)
    print(logits.shape)
    print(targets.shape)
    print(model.loss)
    print(model.loss(y_pred=logits, y_true=targets))
    print(tf.reduce_sum(masked_sequence_loss(y_pred=logits, y_true=targets)))
    break

64
(64, 50, 10036)
(3200, 10036)
(3200, 1)
tf.Tensor(4.362302, shape=(), dtype=float32)
tf.Tensor(4.362302, shape=(), dtype=float32)


In [6]:
def gen_inflect_from_vocab(vocab_file, freq_threshold=1000):
    vbp = {}
    vbz = {}
    nn = {}
    nns = {}
    from_pos = {'NNS': nns, 'NN': nn, 'VBP': vbp, 'VBZ': vbz}

    for line in open(vocab_file):
        if line.startswith(' '):   # empty string token
            continue
        word, pos, count = line.strip().split()
        count = int(count)
        if len(word) > 1 and pos in from_pos and count >= freq_threshold:
            from_pos[pos][word] = count

    verb_infl = {'VBP': 'VBZ', 'VBZ': 'VBP'}
    for word, count in vbz.items():
        candidate = infl_eng.plural_verb(word)
        if candidate in vbp:
            verb_infl[candidate] = word
            verb_infl[word] = candidate

    noun_infl = {'NN': 'NNS', 'NNS': 'NN'}
    for word, count in nn.items():
        candidate = infl_eng.plural_noun(word)
        if candidate in nns:
            noun_infl[candidate] = word
            noun_infl[word] = candidate

    return verb_infl, noun_infl

In [7]:
from util import inflect

infl_eng = inflect.engine()

dependency_fields = ['sentence', 'orig_sentence', 'pos_sentence',
                     'subj', 'verb', 'subj_pos', 'has_rel', 'has_nsubj',
                     'verb_pos', 'subj_index', 'verb_index', 'n_intervening',
                     'last_intervening', 'n_diff_intervening', 'distance',
                     'max_depth', 'all_nouns', 'nouns_up_to_verb']

verb_infl, noun_infl = gen_inflect_from_vocab('wiki.vocab')

In [ ]:
distance_hits = Counter()
distance_total = Counter()
diff_hits = Counter()
diff_total = Counter()

test_data = task.databuilder.as_dataset(split='test', batch_size=1000)
e = 0
for example in tqdm(test_data):
    e += 1
    encoded_sentences = example['sentence']
    s_shape = tf.shape(encoded_sentences)
    batch_size, length = s_shape[0], s_shape[1]
    bos = tf.ones((batch_size,1), dtype=tf.int64) * task.databuilder.sentence_encoder().encode(constants.bos)
    eos = tf.ones((batch_size,1), dtype=tf.int64) * task.databuilder.sentence_encoder().encode(constants.eos)

    encoded_sentences = tf.concat([bos, encoded_sentences, eos], axis=1)
    
    actual_verbs = example['verb']
    inflected_verbs = [verb_infl[v.decode("utf-8")] for v in actual_verbs.numpy()]
    verb_indexes = example['verb_position'] - 1
    distances = example['distance'].numpy()
    nz = example['n_intervening'].numpy()
    n_diffs = example['n_diff_intervening'].numpy()
    
    sentence =  task.databuilder.sentence_encoder().decode(encoded_sentences[0])
    actual_verb_indexes = [task.databuilder.sentence_encoder().encode(v)[0] for v in actual_verbs.numpy()]
    inflected_verb_indexes = [task.databuilder.sentence_encoder().encode(v)[0] for v in inflected_verbs]

    
    scores = model(encoded_sentences)
    actual_batch_indexes = [ (i,verb_indexes[i], actual_verb_indexes[i]) for i in range(len(verb_indexes))]
    actual_scores = tf.compat.v2.gather_nd(scores, actual_batch_indexes)

    inflected_batch_indexes = [ (i,verb_indexes[i], inflected_verb_indexes[i]) for i in range(len(verb_indexes))]
    infelected_scores = tf.compat.v2.gather_nd(scores, inflected_batch_indexes)
    
    corrects = actual_scores > infelected_scores
    for i, c in enumerate(corrects):
        if verb_indexes[i] == 10035:
            continue
        if nz[i] > 4 or distances[i] > 16:
            continue
            
        distance_total[distances[i]] += 1
        distance_hits[distances[i]] += int(c)
        if nz[i] == n_diffs[i]:
            n = nz[i]
            diff_total[n] += 1
            diff_hits[n] += int(c)
            

0it [00:00, ?it/s]

1it [00:01,  1.62s/it]

2it [00:03,  1.63s/it]

4it [00:05,  1.45s/it]

688it [12:58,  1.12s/it]

In [10]:
dis_acc = {}
dis_acc = np.zeros(17)
dif_acc = np.zeros(5)
print('Accuracy by distance')
for k in sorted(distance_hits.keys()):
    v = distance_hits[k]
    acc = v / distance_total[k]
    dis_acc[k] = acc
    print("%d | %.2f" % (k, acc), distance_total[k])

print('Accuracy by intervenings')
for k in sorted(diff_hits.keys()):
    v = diff_hits[k]
    acc = v * 1./diff_total[k]
    print("%d | %.2f" % (k, acc), diff_total[k])
    dif_acc[k] = acc

stats = {'distance': dis_acc, 'intervenings': dif_acc}

Accuracy by distance
1 | 0.72 926851
2 | 0.91 86574
3 | 0.79 71319
4 | 0.84 107576
5 | 0.87 59136
6 | 0.84 37045
7 | 0.83 28371
8 | 0.82 20879
9 | 0.81 14993
10 | 0.80 11346
11 | 0.79 8800
12 | 0.79 6395
13 | 0.77 4950
14 | 0.76 3576
15 | 0.75 2629
16 | 0.74 2001
Accuracy by intervenings
0 | 0.74 1133182
1 | 0.78 51596
2 | 0.79 9045
3 | 0.75 1828
4 | 0.71 460


In [63]:
task.databuilder.sentence_encoder().encode("unk")

[10035]

In [11]:
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

d = dis_acc
lists = sorted(d.items()) # sorted by key, return a list of tuples
x, y = zip(*lists) # unpack a list of pairs into two tuples

plt.plot(x, y)

ModuleNotFoundError: No module named 'seaborn'